## VMAF

In [1]:
import os
import json
import subprocess

folder = "test_vids/encoded"
source = "smithy"
filepath = "rd_results/rd_" + source + ".json"
path_ref = "test_vids/downsampled/" + source + "_3840.yuv"

for file in os.listdir(folder):
    path_dist = os.path.join(folder, file)
    print(path_dist)
    c_vmaf = f"ffmpeg -f rawvideo -r 50 -video_size 3840x2160 -i {path_ref} \
            -i {path_dist} \
            -lavfi \"[0:v]setpts=PTS-STARTPTS[ref];\
                     [1:v]scale=3840x2160:flags=bicubic, setpts=PTS-STARTPTS[dist];\
                     [dist][ref]libvmaf=feature=name=psnr|name=float_ssim|name=float_ms_ssim:log_path=rd_results/vmaf_logs.json:log_fmt=json\" \
            -f null -"
    subprocess.call(c_vmaf, shell=True)
    
    with open("rd_results/vmaf_logs.json", 'r') as r:
        i_data = json.load(r)

    vmaf = i_data["pooled_metrics"]["vmaf"]["mean"]
    ssim = i_data["pooled_metrics"]["float_ssim"]["mean"]
    ms_ssim = i_data["pooled_metrics"]["float_ms_ssim"]["mean"]
    psnr = (6*i_data["pooled_metrics"]["psnr_y"]["mean"] + i_data["pooled_metrics"]["psnr_cb"]["mean"] + i_data["pooled_metrics"]["psnr_cr"]["mean"])/8
    print(vmaf)
    command = "ffprobe -v error -select_streams v:0 -show_entries format:stream -print_format json " + path_dist
    cout = subprocess.run(command.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout
    
    dict = json.loads(cout)  # Convert data from JSON string to dictionary
    file_name = str(dict['format']['filename']).split("/")[-1]
    codec = str(dict['streams'][0]['codec_name']) 
    resolution = int(dict['streams'][0]['bit_rate']) 
    width = int(dict['streams'][0]['width'])
    height = int(dict['streams'][0]['height']) 
    fps = int(str(dict['streams'][0]['r_frame_rate']).split("/")[0])
    cbr = int(file_name[:-4].split("_cf")[1])
    bitrate = int(dict['streams'][0]['bit_rate']) 

    with open(filepath, 'r') as f:
        o_data = json.load(f)
    o_data.append({
        "video_content": source,
        "file_name": file_name,
        "codec": codec,
        "width": width,
        "height": height,
        "framerate": fps,
        "cbr": cbr,
        "bitrate": bitrate,
        "vmaf": vmaf,
        "psnr": psnr,
        "ssim": ssim,
        "ms_ssim": ms_ssim
    })
    with open(filepath, 'w') as w:
        json.dump(o_data, w, indent=4, separators=(',',': '))
        print("written")

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

Output #0, null, to 'pipe:':
  Metadata:
    encoder         : Lavf59.22.100
  Stream #0:0: Video: wrapped_avframe, yuv420p(tv, progressive), 3840x2160, q=2-31, 200 kb/s, 50 fps, 50 tbn
    Metadata:
      encoder         : Lavc59.26.100 wrapped_avframe
frame= 1500 fps=0.5 q=-0.0 Lsize=N/A time=00:00:30.00 bitrate=N/A speed=0.00946x    
video:691kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
[Parsed_libvmaf_3 @ 0x5584b5989940] VMAF score: 38.697556
ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable

Input #1, mov,mp4,m4a,3gp,3g2,mj2, from 'test_vids/encoded/smithyAVC_2560_cf11.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
  Duration: 00:00:30.00, start: 0.000000, bitrate: 69130 kb/s
  Stream #1:0[0x1](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 2560x1440, 69125 kb/s, 50 fps, 50 tbr, 12800 tbn (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
Stream mapping:
  Stream #0:0 (rawvideo) -> setpts:default
  Stream #1:0 (h264) -> scale:default
  libvmaf:default -> Stream #0:0 (wrapped_avframe)
Press [q] to stop, [?] for help
Output #0, null, to 'pipe:':
  Metadata:
    encoder         : Lavf59.22.100
  Stream #0:0: Video: wrapped_avframe, yuv420p(tv, progressive), 3840x2160, q=2-31, 200 kb/s, 50 fps, 50 tbn
    Metadata:
      encoder         : Lavc59.26.100 wrapped_avframe
frame= 1500 fps=0.5 q=-0.0 Lsize=N/A time=00

Output #0, null, to 'pipe:':
  Metadata:
    encoder         : Lavf59.22.100
  Stream #0:0: Video: wrapped_avframe, yuv420p(tv, progressive), 3840x2160, q=2-31, 200 kb/s, 50 fps, 50 tbn
    Metadata:
      encoder         : Lavc59.26.100 wrapped_avframe
frame= 1500 fps=0.5 q=-0.0 Lsize=N/A time=00:00:30.00 bitrate=N/A speed=0.00962x    
video:691kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
[Parsed_libvmaf_3 @ 0x564964673b00] VMAF score: 82.912676
ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable

KeyboardInterrupt: 

In [5]:
import os
import json
import subprocess

folder = "test_vids/encoded"
source = "smithy"
filepath = "rd_results/rd_" + source + ".json"
path_ref = "test_vids/downsampled/" + source + "_3840.yuv"

path_dist = "test_vids/encoded/smithyAVC_640_cf34.mp4"
print(path_dist)
c_vmaf = f"ffmpeg -f rawvideo -r 50 -video_size 3840x2160 -i {path_ref} \
        -i {path_dist} \
        -lavfi \"[0:v]setpts=PTS-STARTPTS[ref];\
                [1:v]scale=3840x2160:flags=bicubic, setpts=PTS-STARTPTS[dist];\
                [dist][ref]libvmaf=feature=name=psnr|name=float_ssim|name=float_ms_ssim:log_path=rd_results/vmaf_logs.json:log_fmt=json\" \
        -f null -"
subprocess.call(c_vmaf, shell=True)

with open("rd_results/vmaf_logs.json", 'r') as r:
    i_data = json.load(r)

vmaf = i_data["pooled_metrics"]["vmaf"]["mean"]
ssim = i_data["pooled_metrics"]["float_ssim"]["mean"]
ms_ssim = i_data["pooled_metrics"]["float_ms_ssim"]["mean"]
psnr = (6*i_data["pooled_metrics"]["psnr_y"]["mean"] + i_data["pooled_metrics"]["psnr_cb"]["mean"] + i_data["pooled_metrics"]["psnr_cr"]["mean"])/8
print(vmaf)
command = "ffprobe -v error -select_streams v:0 -show_entries format:stream -print_format json " + path_dist
cout = subprocess.run(command.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout

dict = json.loads(cout)  # Convert data from JSON string to dictionary
file_name = str(dict['format']['filename']).split("/")[-1]
codec = str(dict['streams'][0]['codec_name']) 
resolution = int(dict['streams'][0]['bit_rate']) 
width = int(dict['streams'][0]['width'])
height = int(dict['streams'][0]['height']) 
fps = int(str(dict['streams'][0]['r_frame_rate']).split("/")[0])
cbr = int(file_name[:-4].split("_cf")[1])
bitrate = int(dict['streams'][0]['bit_rate'])

with open(filepath, 'r') as f:
    o_data = json.load(f)
o_data.append({
    "video_content": source,
    "file_name": file_name,
    "codec": codec,
    "width": width,
    "height": height,
    "framerate": fps,
    "cbr": cbr,
    "bitrate": bitrate,
    "vmaf": vmaf,
    "psnr": psnr,
    "ssim": ssim,
    "ms_ssim": ms_ssim
})
with open(filepath, 'w') as w:
    json.dump(o_data, w, indent=4, separators=(',',': '))
    print("written")

test_vids/encoded/smithyAVC_640_cf34.mp4


ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

0.044292
written


frame= 1500 fps=0.5 q=-0.0 Lsize=N/A time=00:00:30.00 bitrate=N/A speed=0.00953x    
video:691kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
[Parsed_libvmaf_3 @ 0x55ad0dfdda00] VMAF score: 0.044292
